In [1]:
!pip install duckdb==1.2.2
import duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 60.1 MB/s eta 0:00:0000:0100:01


In [3]:
import duckdb
import pandas as pd
from pathlib import Path

csv_dir = Path("hw2_data/")
db_path = "mimic.duckdb"
conn = duckdb.connect(db_path)

csv_files = {
    "ADMISSIONS.csv": "admissions",
    "ICUSTAYS.csv": "icustays",
    "PATIENTS.csv": "patients",
    "PROCEDURES_ICD.csv": "procedures_icd",
    "D_ICD_PROCEDURES.csv": "d_icd_procedures",
    "DRGCODES.csv": "drgcodes",
    "PRESCRIPTIONS.csv": "prescriptions"
}

for csv_name, table_name in csv_files.items():
    csv_path = csv_dir / csv_name
    print(f"Importing {table_name} from {csv_path}")
    conn.execute(f"""
        CREATE OR REPLACE TABLE {table_name} AS 
        SELECT * FROM read_csv_auto('{csv_path.as_posix()}')
    """)


Importing admissions from hw2_data/ADMISSIONS.csv
Importing icustays from hw2_data/ICUSTAYS.csv
Importing patients from hw2_data/PATIENTS.csv
Importing procedures_icd from hw2_data/PROCEDURES_ICD.csv
Importing d_icd_procedures from hw2_data/D_ICD_PROCEDURES.csv
Importing drgcodes from hw2_data/DRGCODES.csv
Importing prescriptions from hw2_data/PRESCRIPTIONS.csv


In [4]:
conn.sql('SHOW TABLES;')

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ admissions       │
│ d_icd_procedures │
│ drgcodes         │
│ icustays         │
│ patients         │
│ prescriptions    │
│ procedures_icd   │
└──────────────────┘

In [5]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'prescriptions';
    """
)

┌───────────────────┬───────────┐
│    column_name    │ data_type │
│      varchar      │  varchar  │
├───────────────────┼───────────┤
│ row_id            │ BIGINT    │
│ subject_id        │ BIGINT    │
│ hadm_id           │ BIGINT    │
│ icustay_id        │ BIGINT    │
│ startdate         │ TIMESTAMP │
│ enddate           │ TIMESTAMP │
│ drug_type         │ VARCHAR   │
│ drug              │ VARCHAR   │
│ drug_name_poe     │ VARCHAR   │
│ drug_name_generic │ VARCHAR   │
│ formulary_drug_cd │ VARCHAR   │
│ gsn               │ VARCHAR   │
│ ndc               │ VARCHAR   │
│ prod_strength     │ VARCHAR   │
│ dose_val_rx       │ VARCHAR   │
│ dose_unit_rx      │ VARCHAR   │
│ form_val_disp     │ VARCHAR   │
│ form_unit_disp    │ VARCHAR   │
│ route             │ VARCHAR   │
├───────────────────┴───────────┤
│ 19 rows             2 columns │
└───────────────────────────────┘

In [6]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'admissions';
    """
)

┌──────────────────────┬───────────┐
│     column_name      │ data_type │
│       varchar        │  varchar  │
├──────────────────────┼───────────┤
│ row_id               │ BIGINT    │
│ subject_id           │ BIGINT    │
│ hadm_id              │ BIGINT    │
│ admittime            │ TIMESTAMP │
│ dischtime            │ TIMESTAMP │
│ deathtime            │ TIMESTAMP │
│ admission_type       │ VARCHAR   │
│ admission_location   │ VARCHAR   │
│ discharge_location   │ VARCHAR   │
│ insurance            │ VARCHAR   │
│ language             │ VARCHAR   │
│ religion             │ VARCHAR   │
│ marital_status       │ VARCHAR   │
│ ethnicity            │ VARCHAR   │
│ edregtime            │ TIMESTAMP │
│ edouttime            │ TIMESTAMP │
│ diagnosis            │ VARCHAR   │
│ hospital_expire_flag │ BIGINT    │
│ has_chartevents_data │ BIGINT    │
├──────────────────────┴───────────┤
│ 19 rows                2 columns │
└──────────────────────────────────┘

In [7]:
conn.sql(
    """
    SELECT
        a.ethnicity,
        p.drug,
        COUNT(*) AS total_prescriptions
    FROM prescriptions p
    JOIN admissions a
        ON p.hadm_id = a.hadm_id
    WHERE p.drug IS NOT NULL
    GROUP BY a.ethnicity, p.drug
    QUALIFY ROW_NUMBER() OVER (PARTITION BY a.ethnicity ORDER BY COUNT(*) DESC) = 1
    ORDER BY a.ethnicity;
    """
)


┌──────────────────────────────────────────────────────────┬──────────────────────┬─────────────────────┐
│                        ethnicity                         │         drug         │ total_prescriptions │
│                         varchar                          │       varchar        │        int64        │
├──────────────────────────────────────────────────────────┼──────────────────────┼─────────────────────┤
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │ 5% Dextrose          │                  27 │
│ ASIAN                                                    │ D5W                  │                  27 │
│ BLACK/AFRICAN AMERICAN                                   │ Insulin              │                  38 │
│ HISPANIC OR LATINO                                       │ 5% Dextrose          │                  28 │
│ HISPANIC/LATINO - PUERTO RICAN                           │ 0.9% Sodium Chloride │                  86 │
│ OTHER                                       

In [8]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'procedures_icd';
    """
)

┌─────────────┬───────────┐
│ column_name │ data_type │
│   varchar   │  varchar  │
├─────────────┼───────────┤
│ row_id      │ BIGINT    │
│ subject_id  │ BIGINT    │
│ hadm_id     │ BIGINT    │
│ seq_num     │ BIGINT    │
│ icd9_code   │ VARCHAR   │
└─────────────┴───────────┘

In [9]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'patients';
    """
)

┌─────────────┬───────────┐
│ column_name │ data_type │
│   varchar   │  varchar  │
├─────────────┼───────────┤
│ row_id      │ BIGINT    │
│ subject_id  │ BIGINT    │
│ gender      │ VARCHAR   │
│ dob         │ TIMESTAMP │
│ dod         │ TIMESTAMP │
│ dod_hosp    │ TIMESTAMP │
│ dod_ssn     │ TIMESTAMP │
│ expire_flag │ BIGINT    │
└─────────────┴───────────┘

In [10]:
conn.sql(
    """
    WITH patient_ages AS (
    SELECT
        p.subject_id,
        FLOOR((CAST(a.admittime AS DATE) - CAST(p.dob AS DATE)) / 365.25) AS age
    FROM patients p
    JOIN admissions a ON p.subject_id = a.subject_id
    ),

    age_groups AS (
        SELECT subject_id,
            CASE 
                WHEN age <= 19 THEN '<=19'
                WHEN age BETWEEN 20 AND 49 THEN '20-49'
                WHEN age BETWEEN 50 AND 79 THEN '50-79'
                ELSE '>=80'
            END AS age_group
        FROM patient_ages
    ),

    procedure_summary AS (
        SELECT
            ag.age_group,
            d.short_title AS procedure_name,
            COUNT(*) AS procedure_count
        FROM procedures_icd pi
        JOIN age_groups ag ON pi.subject_id = ag.subject_id
        LEFT JOIN d_icd_procedures d ON pi.icd9_code = d.icd9_code
        GROUP BY ag.age_group, procedure_name
    )
    
    SELECT *
    FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY procedure_count DESC) AS rank
        FROM procedure_summary
    )
    WHERE rank <= 3
    ORDER BY age_group, rank;
    """
)

┌───────────┬──────────────────────────┬─────────────────┬───────┐
│ age_group │      procedure_name      │ procedure_count │ rank  │
│  varchar  │         varchar          │      int64      │ int64 │
├───────────┼──────────────────────────┼─────────────────┼───────┤
│ 20-49     │ Venous cath NEC          │              11 │     1 │
│ 20-49     │ Entral infus nutrit sub  │              11 │     2 │
│ 20-49     │ Insert endotracheal tube │               9 │     3 │
│ 50-79     │ Venous cath NEC          │             185 │     1 │
│ 50-79     │ Entral infus nutrit sub  │             170 │     2 │
│ 50-79     │ Insert endotracheal tube │              51 │     3 │
│ <=19      │ Venous cath NEC          │               3 │     1 │
│ <=19      │ Skin closure NEC         │               2 │     2 │
│ <=19      │ Percu endosc gastrostomy │               1 │     3 │
│ >=80      │ Venous cath NEC          │              22 │     1 │
│ >=80      │ Packed cell transfusion  │              16 │    

In [11]:
conn.sql(
    """
    WITH icu_los AS (
        SELECT
            icustays.subject_id,
            icustays.hadm_id,
            EXTRACT(EPOCH FROM icustays.outtime - icustays.intime) / 3600.0 AS los_hours
        FROM icustays
    ),

    joined AS (
        SELECT
            los_hours / 24.0 AS los_days,
            patients.gender,
            admissions.ethnicity
        FROM icu_los
        JOIN patients ON icu_los.subject_id = patients.subject_id
        JOIN admissions ON icu_los.hadm_id = admissions.hadm_id
    )

    SELECT 
        gender,
        ethnicity,

        ROUND(AVG(los_days), 1) AS avg_icu_days
    
    FROM joined
    GROUP BY gender, ethnicity
    ORDER BY gender, ethnicity;
    """
    )



┌─────────┬──────────────────────────────────────────────────────────┬──────────────┐
│ gender  │                        ethnicity                         │ avg_icu_days │
│ varchar │                         varchar                          │    double    │
├─────────┼──────────────────────────────────────────────────────────┼──────────────┤
│ F       │ ASIAN                                                    │          0.7 │
│ F       │ BLACK/AFRICAN AMERICAN                                   │         11.2 │
│ F       │ HISPANIC OR LATINO                                       │          7.5 │
│ F       │ OTHER                                                    │          1.3 │
│ F       │ UNKNOWN/NOT SPECIFIED                                    │          5.5 │
│ F       │ WHITE                                                    │          5.2 │
│ M       │ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │         11.3 │
│ M       │ ASIAN                                     

In [12]:
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1468  100  1468    0     0   6095      0 --:--:-- --:--:-- --:--:--  6091


In [13]:
%pip install cassandra-sigv4
%pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 75.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

/tmp/ipykernel_523/332746716.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [15]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [16]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [17]:
# establishing connection to Keyspace
session = cluster.connect()

In [18]:
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='aet7207_hw2', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='can1469_hw02', durable_writes=True, replication=OrderedMapSe

In [19]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS mimic
    WITH replication = {
        'class': 'SingleRegionStrategy'
    };
""")

In [20]:
session.set_keyspace('mimic')

In [23]:
import os
import pandas as pd
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel

In [24]:
csv_folder = 'hw2_data'  # Adjust if your folder is in a different location
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

dfs = {}
for file in csv_files:
    path = os.path.join(csv_folder, file)
    df = pd.read_csv(path)
    table_name = os.path.splitext(file)[0].lower().replace("-", "_").replace(" ", "_")
    dfs[table_name] = df


In [25]:
def cassandra_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "DOUBLE"
    else:
        return "TEXT"

for table_name, df in dfs.items():
    if df.empty:
        print(f"Skipping empty file: {table_name}")
        continue

    column_defs = ",\n    ".join(
        f"{col.lower().replace(' ', '_')} {cassandra_type(df[col])}" for col in df.columns
    )

    primary_key = df.columns[0].lower().replace(" ", "_")  # basic default; change if needed

    create_stmt = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {column_defs},
        PRIMARY KEY ({primary_key})
    );
    """
    
    session.execute(SimpleStatement(create_stmt, consistency_level=ConsistencyLevel.LOCAL_QUORUM))
    print(f"Created table: {table_name}")


Created table: drgcodes
Created table: icustays
Created table: patients
Created table: procedures_icd
Created table: admissions
Created table: d_icd_procedures
Created table: prescriptions


In [26]:
for table_name, df in dfs.items():
    if df.empty:
        continue

    df.columns = [col.lower().replace(" ", "_") for col in df.columns]
    col_names = ", ".join(df.columns)
    placeholders = ", ".join(["%s"] * len(df.columns))
    
    insert_stmt = SimpleStatement(
        f"INSERT INTO {table_name} ({col_names}) VALUES ({placeholders})",
        consistency_level=ConsistencyLevel.LOCAL_QUORUM
    )

    for _, row in df.iterrows():
        values = [None if pd.isna(v) else v for v in row.tolist()]
        session.execute(insert_stmt, tuple(values))
    
    print(f"Inserted data into: {table_name}")


Inserted data into: drgcodes
Inserted data into: icustays
Inserted data into: patients
Inserted data into: procedures_icd
Inserted data into: admissions
Inserted data into: d_icd_procedures
Inserted data into: prescriptions


In [27]:
rows = session.execute(f"SELECT * FROM {list(dfs.keys())[0]} LIMIT 5;")
for row in rows:
    print(row)


Row(row_id=75209, description='CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC', drg_code=191, drg_mortality=None, drg_severity=None, drg_type='MS', hadm_id=128293, subject_id=42275)
Row(row_id=66128, description='Head Trauma w/ Coma > 1 Hr or Hemorrhage', drg_code=553, drg_mortality=3.0, drg_severity=3.0, drg_type='APR ', hadm_id=186071, subject_id=43779)
Row(row_id=7125, description='SIMPLE PNEUMONIA & PLEURISY AGE >17 WITH COMPLICATIONS, COMORBIDITIES', drg_code=89, drg_mortality=None, drg_severity=None, drg_type='HCFA', hadm_id=105150, subject_id=10117)
Row(row_id=59531, description='Heart Failure', drg_code=1942, drg_mortality=2.0, drg_severity=2.0, drg_type='APR ', hadm_id=161765, subject_id=40456)
Row(row_id=125347, description='Septicemia & Disseminated Infections', drg_code=7204, drg_mortality=4.0, drg_severity=4.0, drg_type='APR ', hadm_id=149469, subject_id=41976)


In [28]:
prescriptions = session.execute("SELECT hadm_id, drug FROM prescriptions;")
admissions = session.execute("SELECT hadm_id, ethnicity FROM admissions;")


In [29]:
hadm_to_ethnicity = {}
for row in admissions:
    if row.ethnicity:
        hadm_to_ethnicity[row.hadm_id] = row.ethnicity


In [30]:
from collections import defaultdict, Counter

ethnicity_drug_counts = defaultdict(Counter)

for row in prescriptions:
    ethnicity = hadm_to_ethnicity.get(row.hadm_id)
    if ethnicity and row.drug:
        ethnicity_drug_counts[ethnicity][row.drug] += 1

# Print top drug per ethnicity
for ethnicity, counter in ethnicity_drug_counts.items():
    top_drug, count = counter.most_common(1)[0]
    print(f"{ethnicity}: {top_drug} ({count} prescriptions)")


AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE: 5% Dextrose (27 prescriptions)
WHITE: Potassium Chloride (381 prescriptions)
HISPANIC/LATINO - PUERTO RICAN: 0.9% Sodium Chloride (86 prescriptions)
BLACK/AFRICAN AMERICAN: Insulin (38 prescriptions)
UNKNOWN/NOT SPECIFIED: D5W (37 prescriptions)
HISPANIC OR LATINO: 5% Dextrose (28 prescriptions)
OTHER: NS (11 prescriptions)
ASIAN: D5W (27 prescriptions)
UNABLE TO OBTAIN: 0.9% Sodium Chloride (28 prescriptions)


In [32]:
patients = session.execute("SELECT subject_id, dob FROM patients;")
subject_to_dob = {row.subject_id: row.dob for row in patients if row.dob}


In [33]:
admissions = session.execute("SELECT subject_id, hadm_id, admittime FROM admissions;")

In [34]:
from datetime import datetime

hadm_to_age_group = {}
for row in admissions:
    dob_str = subject_to_dob.get(row.subject_id)
    adm_str = row.admittime
    if dob_str and adm_str:
        try:
            dob = datetime.strptime(dob_str, "%Y-%m-%d %H:%M:%S")
            admittime = datetime.strptime(adm_str, "%Y-%m-%d %H:%M:%S")
            age = (admittime - dob).days // 365
            if age <= 19:
                group = "0–19"
            elif age <= 49:
                group = "20–49"
            elif age <= 79:
                group = "50–79"
            else:
                group = "80+"
            hadm_to_age_group[row.hadm_id] = group
        except ValueError as e:
            print(f"Skipping row due to date parse error: {e}")

In [35]:
procedures = session.execute("SELECT hadm_id, icd9_code FROM procedures_icd;")


In [36]:
codes = session.execute("SELECT icd9_code, long_title FROM d_icd_procedures;")
icd_map = {row.icd9_code: row.long_title for row in codes}


In [38]:
from collections import defaultdict, Counter

agegroup_proc_counts = defaultdict(Counter)

for row in procedures:
    age_group = hadm_to_age_group.get(row.hadm_id)
    proc = icd_map.get(row.icd9_code, row.icd9_code)
    if age_group and proc:
        agegroup_proc_counts[age_group][proc] += 1

# Print top 3 per age group
for group, counter in agegroup_proc_counts.items():
    print(f"\nAge group {group}:")
    for proc, count in counter.most_common(3):
        print(f"  {proc} — {count} procedures")




Age group 50–79:
  Venous catheterization, not elsewhere classified — 26 procedures
  Enteral infusion of concentrated nutritional substances — 22 procedures
  Transfusion of packed cells — 13 procedures

Age group 20–49:
  Venous catheterization, not elsewhere classified — 9 procedures
  Enteral infusion of concentrated nutritional substances — 7 procedures
  Percutaneous abdominal drainage — 6 procedures

Age group 80+:
  Venous catheterization, not elsewhere classified — 19 procedures
  Transfusion of packed cells — 13 procedures
  Insertion of endotracheal tube — 8 procedures

Age group 0–19:
  Venous catheterization, not elsewhere classified — 2 procedures
  Other skeletal traction — 1 procedures
  Percutaneous [endoscopic] gastrostomy [PEG] — 1 procedures


In [39]:
icustays = session.execute("SELECT subject_id, intime, outtime FROM icustays;")
stay_lengths = defaultdict(list)
for row in icustays:
    if row.intime and row.outtime:
        try:
            intime = datetime.strptime(row.intime, "%Y-%m-%d %H:%M:%S")
            outtime = datetime.strptime(row.outtime, "%Y-%m-%d %H:%M:%S")
            delta = outtime - intime
            stay_days = delta.days + delta.seconds / 86400
            stay_lengths[row.subject_id].append(stay_days)
        except ValueError as e:
            print(f"Skipping row due to date parse error: {e}")

In [40]:
patients = session.execute("SELECT subject_id, gender FROM patients;")
genders = {row.subject_id: row.gender for row in patients}

admissions = session.execute("SELECT subject_id, ethnicity FROM admissions;")
ethnicities = {}
for row in admissions:
    if row.subject_id not in ethnicities and row.ethnicity:
        ethnicities[row.subject_id] = row.ethnicity  # only keep first seen


In [41]:
import statistics
gender_stays = defaultdict(list)
ethnicity_stays = defaultdict(list)

for subject_id, stays in stay_lengths.items():
    avg_stay = statistics.mean(stays)
    gender = genders.get(subject_id)
    ethnicity = ethnicities.get(subject_id)
    if gender:
        gender_stays[gender].append(avg_stay)
    if ethnicity:
        ethnicity_stays[ethnicity].append(avg_stay)

# Report
print("Average ICU stay by gender:")
for gender, stays in gender_stays.items():
    print(f"{gender}: {statistics.mean(stays):.2f} days")

print("\nAverage ICU stay by ethnicity:")
for eth, stays in ethnicity_stays.items():
    print(f"{eth}: {statistics.mean(stays):.2f} days")


Average ICU stay by gender:
M: 3.38 days
F: 5.69 days

Average ICU stay by ethnicity:
WHITE: 4.22 days
UNKNOWN/NOT SPECIFIED: 5.19 days
OTHER: 0.93 days
BLACK/AFRICAN AMERICAN: 8.28 days
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE: 11.34 days
HISPANIC/LATINO - PUERTO RICAN: 3.24 days
HISPANIC OR LATINO: 6.48 days
ASIAN: 3.89 days
UNABLE TO OBTAIN: 13.36 days
